Task 1: Regresión Lineal 

1.1 El coeficiente “3w2” representa cuanto cambia la variable dependiente “y” por cada unidad adicional de la característica "X2”. Siendo más específicos, por cada incremento de 1 en “X2”, “y "aumentara “3W2”
1.2 La multicolinealidad es cuando dos o más variables independientes están altamente relacionadas entre sí, causando que sea más dificultosa la tarea de saber cuál de ellos es realmente el responsable del cambio 


#TASK 2.0 Preparacion de dataset

Exploracion de datos inicial

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

filepath = "dataset_phishing.csv"
dataset = pd.read_csv(filepath)
#Exploracion de datos - Inciso 2.0
print("Distribución inicial de la columna 'status':")
print(dataset['status'].value_counts())
print("\nVerificacion de datos nulos")
print(dataset.isnull().sum())
print("\nResumen estadistico de los datos")
print(dataset.describe())




Distribución inicial de la columna 'status':
status
legitimate    5715
phishing      5715
Name: count, dtype: int64

Verificacion de datos nulos
url                0
length_url         0
length_hostname    0
ip                 0
nb_dots            0
                  ..
web_traffic        0
dns_record         0
google_index       0
page_rank          0
status             0
Length: 89, dtype: int64

Resumen estadistico de los datos
         length_url  length_hostname            ip       nb_dots  \
count  11430.000000     11430.000000  11430.000000  11430.000000   
mean      61.126684        21.090289      0.150569      2.480752   
std       55.297318        10.777171      0.357644      1.369686   
min       12.000000         4.000000      0.000000      1.000000   
25%       33.000000        15.000000      0.000000      2.000000   
50%       47.000000        19.000000      0.000000      2.000000   
75%       71.000000        24.000000      0.000000      3.000000   
max     1641.000000  

Encoding de las columnas (Unicamente de la columna status)

In [2]:
#Encoding de las columnas
# Ignorar la columna URL
if 'url' in dataset.columns:
    dataset = dataset.drop(columns=['url'])
# Codificar la columna status
if 'status' in dataset.columns:
    label_encoder = LabelEncoder()
    dataset['status'] = label_encoder.fit_transform(dataset['status'])

# Mostrar las primeras filas del dataset modificado
dataset.head(), dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 88 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   length_url                  11430 non-null  int64  
 1   length_hostname             11430 non-null  int64  
 2   ip                          11430 non-null  int64  
 3   nb_dots                     11430 non-null  int64  
 4   nb_hyphens                  11430 non-null  int64  
 5   nb_at                       11430 non-null  int64  
 6   nb_qm                       11430 non-null  int64  
 7   nb_and                      11430 non-null  int64  
 8   nb_or                       11430 non-null  int64  
 9   nb_eq                       11430 non-null  int64  
 10  nb_underscore               11430 non-null  int64  
 11  nb_tilde                    11430 non-null  int64  
 12  nb_percent                  11430 non-null  int64  
 13  nb_slash                    114

(   length_url  length_hostname  ip  nb_dots  nb_hyphens  nb_at  nb_qm  nb_and  \
 0          37               19   0        3           0      0      0       0   
 1          77               23   1        1           0      0      0       0   
 2         126               50   1        4           1      0      1       2   
 3          18               11   0        2           0      0      0       0   
 4          55               15   0        2           2      0      0       0   
 
    nb_or  nb_eq  ...  domain_in_title  domain_with_copyright  \
 0      0      0  ...                0                      1   
 1      0      0  ...                1                      0   
 2      0      3  ...                1                      0   
 3      0      0  ...                1                      0   
 4      0      0  ...                0                      1   
 
    whois_registered_domain  domain_registration_length  domain_age  \
 0                        0                

Revision de balanceo de dataset

In [3]:
# Verificar la distribución de las clases en la columna 'status'
class_distribution = dataset['status'].value_counts()
class_distribution_percentage = dataset['status'].value_counts(normalize=True) * 100

class_distribution, class_distribution_percentage


(status
 0    5715
 1    5715
 Name: count, dtype: int64,
 status
 0    50.0
 1    50.0
 Name: proportion, dtype: float64)

Verificacion si es necesario el escalamiento de variables

In [4]:
# Calcular el rango (mínimo y máximo) de las características numéricas
feature_ranges = dataset.drop(columns=['status']).agg(['min', 'max']).T
feature_ranges['range'] = feature_ranges['max'] - feature_ranges['min']
feature_ranges_sorted = feature_ranges.sort_values('range', ascending=False)
feature_ranges_sorted.head()




,min,max,range
web_traffic,0.0,10767986.0,10767986.0
domain_registration_length,-1.0,29829.0,29830.0
domain_age,-12.0,12874.0,12886.0
nb_hyperlinks,0.0,4659.0,4659.0
length_url,12.0,1641.0,1629.0


Escalamiento de variables

In [5]:
from sklearn.preprocessing import StandardScaler

# Separar las características y la etiqueta
X = dataset.drop(columns=['status'])
y = dataset['status']

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convertir X_scaled de nuevo a un DataFrame para mantener las columnas originales
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Mostrar las primeras filas del dataset escalado
X_scaled_df.head()


,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank
0,-0.436327,-0.193964,-0.421020,0.379116,-0.477984,-0.142915,-0.387464,-0.197604,0.0,-0.293683,...,-0.377549,-1.860473,1.129194,-0.28037,-0.549299,-1.307594,-0.429340,6.978227,0.934264,0.320974
1,0.287067,0.177207,2.375182,-1.081136,-0.477984,-0.142915,-0.387464,-0.197604,0.0,-0.293683,...,-0.377549,0.537498,-0.885587,-0.28037,-0.510022,0.548471,-0.429340,-0.143303,0.934264,-0.467407
2,1.173224,2.682613,2.375182,1.109242,0.001174,-0.142915,2.356473,2.237556,0.0,2.711505,...,-0.377549,0.537498,-0.885587,-0.28037,-0.587348,-0.018839,2.491612,-0.143303,0.934264,-1.255788
3,-0.779940,-0.936306,-0.421020,-0.351010,-0.477984,-0.142915,-0.387464,-0.197604,0.0,-0.293683,...,-0.377549,0.537498,-0.885587,-0.28037,-0.528433,-1.307594,-0.375359,-0.143303,-1.070361,-0.073217
4,-0.110800,-0.565135,-0.421020,-0.351010,0.480332,-0.142915,-0.387464,-0.197604,0.0,-0.293683,...,-0.377549,-1.860473,1.129194,-0.28037,-0.329595,1.323334,-0.424968,-0.143303,-1.070361,1.109355


In [6]:
from sklearn.model_selection import train_test_split

# Dividir el dataset en 80% entrenamiento y 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42, stratify=y)

# Mostrar la distribución de clases en los conjuntos
train_distribution = y_train.value_counts(normalize=True) * 100
test_distribution = y_test.value_counts(normalize=True) * 100

X_train.shape, X_test.shape, train_distribution, test_distribution


((9144, 87),
 (2286, 87),
 status
 0    50.0
 1    50.0
 Name: proportion, dtype: float64,
 status
 1    50.0
 0    50.0
 Name: proportion, dtype: float64)

Task 2.1 - Regresión Logística 



In [8]:
# Función personalizada para calcular el logaritmo natural
def log(x, iterations=100):
    """Aproximación del logaritmo natural usando la serie de Taylor."""
    if x <= 0:
        raise ValueError("El logaritmo no está definido para valores <= 0")
    n = (x - 1) / (x + 1)  # Transformación para mejorar la convergencia
    result = 0
    for i in range(1, iterations * 2, 2):  # Serie de Taylor para ln(x)
        result += (1 / i) * (n ** i)
    return 2 * result

# Función sigmoide
def sigmoid(z):
    """Calcula la función sigmoide."""
    return 1 / (1 + (2.718281828459045 ** -z))  # Usando e constante

# Función de costo
def compute_cost(X, y, theta):
    """Calcula el costo logístico sin regularización."""
    m = len(y)
    cost = 0
    for i in range(m):
        z = sum(X[i][j] * theta[j] for j in range(len(theta)))
        h = sigmoid(z)
        cost += -y[i] * log(h) - (1 - y[i]) * log(1 - h)
    return cost / m

# Gradiente descendente
def gradient_descent(X, y, theta, learning_rate, epochs):
    """Implementa gradiente descendente para optimizar los pesos."""
    m = len(y)
    cost_history = []

    for _ in range(epochs):
        gradients = [0] * len(theta)
        for i in range(m):
            z = sum(X[i][j] * theta[j] for j in range(len(theta)))
            h = sigmoid(z)
            for j in range(len(theta)):
                gradients[j] += (h - y[i]) * X[i][j]
        for j in range(len(theta)):
            theta[j] -= learning_rate * gradients[j] / m
        # Calcular costo para guardar en historial
        cost_history.append(compute_cost(X, y, theta))
    
    return theta, cost_history

# Generar datos sintéticos
import random
random.seed(42)

num_samples = 300
X = []
y = []

for _ in range(num_samples):
    x1 = random.gauss(2, 1)
    x2 = random.gauss(-2, 1)
    label = 1 if x1 + x2 > 0 else 0
    X.append([1, x1, x2])  # Agregar el término de sesgo (1)
    y.append(label)

# Inicializar parámetros
theta = [0] * len(X[0])
learning_rate = 0.01
epochs = 1000

# Entrenar el modelo
theta, cost_history = gradient_descent(X, y, theta, learning_rate, epochs)

# Visualizar la evolución del costo
print("Evolución del costo por épocas:")
for epoch, cost in enumerate(cost_history[:10]):  # Mostrar las primeras 10 épocas
    print(f"Época {epoch + 1}: Costo = {cost:.4f}")

# Mostrar los parámetros finales
print("\nParámetros finales del modelo:")
for i, param in enumerate(theta):
    print(f"Theta {i}: {param:.4f}")

# Dibujar los resultados (opcional)
print("\nDatos clasificados:")
for i in range(10):  # Mostrar 10 ejemplos clasificados
    z = sum(X[i][j] * theta[j] for j in range(len(theta)))
    h = sigmoid(z)
    prediccion = 1 if h >= 0.5 else 0
    print(f"Real: {y[i]}, Predicción: {prediccion}, Probabilidad: {h:.4f}")


Evolución del costo por épocas:
Época 1: Costo = 0.6916
Época 2: Costo = 0.6901
Época 3: Costo = 0.6886
Época 4: Costo = 0.6871
Época 5: Costo = 0.6856
Época 6: Costo = 0.6841
Época 7: Costo = 0.6826
Época 8: Costo = 0.6811
Época 9: Costo = 0.6797
Época 10: Costo = 0.6782

Parámetros finales del modelo:
Theta 0: -0.0316
Theta 1: 1.2960
Theta 2: 1.3510

Datos clasificados:
Real: 0, Predicción: 0, Probabilidad: 0.3631
Real: 1, Predicción: 1, Probabilidad: 0.6598
Real: 0, Predicción: 0, Probabilidad: 0.0887
Real: 1, Predicción: 0, Probabilidad: 0.4820
Real: 0, Predicción: 0, Probabilidad: 0.4338
Real: 1, Predicción: 1, Probabilidad: 0.8496
Real: 1, Predicción: 1, Probabilidad: 0.7024
Real: 0, Predicción: 0, Probabilidad: 0.0780
Real: 1, Predicción: 1, Probabilidad: 0.8753
Real: 0, Predicción: 0, Probabilidad: 0.4425
